In [2]:
import numpy as np
import random

In [3]:
tiles = {}
tiles['A'] = {'count':9,'points':1}
tiles['B'] = {'count':2,'points':3}
tiles['C'] = {'count':2,'points':3}
tiles['D'] = {'count':4,'points':2}
tiles['E'] = {'count':12,'points':1}
tiles['F'] = {'count':2,'points':4}
tiles['G'] = {'count':3,'points':2}
tiles['H'] = {'count':2,'points':4}
tiles['I'] = {'count':9,'points':1}
tiles['J'] = {'count':1,'points':8}
tiles['K'] = {'count':1,'points':5}
tiles['L'] = {'count':4,'points':1}
tiles['M'] = {'count':2,'points':3}
tiles['N'] = {'count':6,'points':1}
tiles['O'] = {'count':8,'points':1}
tiles['P'] = {'count':2,'points':3}
tiles['Q'] = {'count':1,'points':10}
tiles['R'] = {'count':6,'points':1}
tiles['S'] = {'count':4,'points':1}
tiles['T'] = {'count':6,'points':1}
tiles['U'] = {'count':4,'points':1}
tiles['V'] = {'count':2,'points':4}
tiles['W'] = {'count':2,'points':4}
tiles['X'] = {'count':1,'points':8}
tiles['Y'] = {'count':2,'points':4}
tiles['Z'] = {'count':1,'points':10}
tiles['1'] = {'count':1,'points':0}
tiles['2'] = {'count':1,'points':0}

In [4]:
dictionary = {}
maxLen = 0
with open('/Users/Joseph/Documents/Puzzles/FiveThirtyEight/Scrabble/enable1.txt','r') as dictFile:
    dictLine = dictFile.readline().strip('\n')
    while dictLine:
        if len(dictLine)>maxLen:
            maxLen = len(dictLine)    
        dictionary[dictLine.upper()] = sum(tiles[c]['points'] for c in dictLine.upper())
        dictLine = dictFile.readline().strip('\n')
        

In [5]:
def gen_rand_string(n):
    strings = []
    base_string = ''.join(k*tiles[k]['count'] for k in tiles.keys())
    base_list = list(base_string)
    for i in range(n):
        x = random.sample(base_list,len(base_list))
        strings.append(x)
    return strings

In [6]:
strs = gen_rand_string(50)
init_pop = []
for s in strs:
    blanks = [chr(c) for c in np.random.randint(65,91,2)]
    init_pop.append((s,blanks))
print init_pop

[(['S', 'A', 'S', '2', 'E', 'A', 'N', 'D', 'T', 'M', 'W', 'R', 'R', 'O', 'P', 'S', 'I', 'A', 'V', 'I', 'N', 'I', 'O', 'I', 'E', 'T', 'P', 'L', 'E', 'N', 'U', 'R', 'E', 'I', 'T', 'U', 'L', 'T', 'O', 'O', 'N', 'A', 'E', 'B', 'X', 'B', 'F', 'R', 'A', 'U', 'T', 'I', 'T', 'K', 'F', 'N', 'S', 'G', 'C', 'A', 'H', 'Y', 'O', 'H', 'V', 'W', 'O', 'R', 'E', 'C', 'O', 'E', 'E', 'D', 'U', 'I', 'A', 'A', 'E', '1', 'D', 'Z', 'M', 'E', 'E', 'L', 'I', 'Y', 'I', 'D', 'Q', 'A', 'G', 'N', 'L', 'G', 'R', 'O', 'J', 'E'], ['F', 'Q']), (['N', 'L', 'S', 'A', 'F', 'Y', 'U', 'O', 'E', 'C', 'E', 'B', 'N', 'U', 'O', 'I', 'A', 'A', 'C', 'T', 'O', 'H', 'U', 'E', 'T', 'A', 'T', 'N', 'G', 'T', 'E', 'E', 'L', 'G', 'T', 'E', 'G', 'R', 'I', 'D', 'F', 'A', '2', 'U', 'V', 'S', 'J', 'M', 'A', 'N', 'Y', 'I', 'E', 'V', 'E', 'I', 'E', 'W', 'O', 'X', 'R', 'A', 'O', 'K', 'Z', 'S', 'N', 'R', 'N', 'E', 'I', 'L', 'R', 'H', 'I', 'I', 'T', 'P', 'Q', 'O', 'D', 'M', 'E', 'O', 'L', 'O', 'D', 'I', 'D', 'I', 'P', 'B', 'E', 'W', 'S', 'R', '

In [14]:
def eval_string(string,b1,b2,maxLen):
    words = set()
    blank1 = set()
    blank2 = set()
    blank12 = set()
    for i in range(len(string)):
        for j in range(min(maxLen,len(string)-i)):
            word = ''.join(string[i:i+j])
            if '1' in word:
                if '2' in word:
                    word.replace('1',b1)
                    word.replace('2',b2)
                    if word in dictionary:
                        blank12.add(word)
                else:
                    word.replace('1',b1)
                    if word in dictionary:
                        blank1.add(word)
            elif '2' in word:
                word.replace('2',b2)
                if word in dictionary:
                    blank2.add(word)
            elif word in dictionary:
                words.add(word)
    
    blank1_val = sum(dictionary[w]-tiles[b1]['points'] for w in blank1)
    blank2_val = sum(dictionary[w]-tiles[b2]['points'] for w in blank2)
    blank12_val = sum(dictionary[w]-tiles[b1]['points']-tiles[b2]['points'] for w in blank12)

    val = sum(dictionary[w] for w in words)
    return val,words

In [15]:
def genetic_algorithm(init_pop,pop_size,n_steps,pnt_mut_rate,swp_mut_rate,blnk_mut_rate,keep_best=True):
    
    def reproduce(scores,k):
        scores = np.array(scores,dtype='float')
        scores -= min(scores)
        scores /= sum(scores)
        bins = np.append(0,np.cumsum(scores))
        r = np.random.rand(k)
        idx = np.digitize(r,bins)-1
        return idx
        
    def pnt_mutate(string,pnt_mut_rate):
        n = len(string)
        p = pnt_mut_rate/float(n)
        r = np.random.rand(n)
        for i in range(n):
            if r[i] < p:
                swp_idx = np.random.randint(n)
                swp = string[i]
                string[i] = string[swp_idx]
                string[swp_idx] = swp
                
        return string
    
    def blnk_mutate(blanks,blnk_mutate_rate):
        for i in range(len(blanks)):
            if np.random.rand()<blnk_mutate_rate:
                blanks[i] = chr(np.random.randint(65,91))
        return blanks
    
    def swp_mutate(s,swp_mut_rate):
        if np.random.rand()<swp_mut_rate:
            idx = sorted(np.random.randint(0,len(s),4))
            s = s[0:idx[0]]+s[idx[2]:idx[3]]+s[idx[1]:idx[2]]+s[idx[0]:idx[1]]+s[idx[3]:]
        return s
    
    def score_pop(population):
        score = []
        for indiv in population:
            string = indiv[0]
            b1 = indiv[1][0]
            b2 = indiv[1][1]
            score.append(eval_string(string,b1,b2,maxLen)[0])
        return score
    
    population = init_pop
    for step in range(n_steps):
        print 'step: ' + str(step)
        new_pop = []
        score = score_pop(population)
        max_val = max(score)
        max_idx = np.where(np.array(score)==max_val)[0]
        print 'max score: '+ str(max_val)
        best_pop = [population[i] for i in max_idx]
        print [''.join(b[0]) for b in best_pop]
        print [b[1] for b in best_pop]
        n_offspring = pop_size
        if keep_best:
            n_offspring -= len(best_pop)
        idx = reproduce(score,n_offspring)
        for i in idx:
            string = population[i][0][:]
            blanks = population[i][1][:]
            string = pnt_mutate(string,pnt_mut_rate)
            string = swp_mutate(string,swp_mut_rate)
            blanks = blnk_mutate(blanks,blnk_mut_rate)
            new_pop.append((string,blanks))
        if keep_best:
            population = best_pop + new_pop
    score = score_pop(population)
    return new_pop,score

In [ ]:
pop,score = genetic_algorithm(init_pop,100,1000,5,0.5,.2)

step: 0
max score: 215
['BEERAJDATPYENGOTASINNAKFOLDATIFLGYHEPTIERXTUAINORIOUBO2RECWMMNCEEG1VEOANIUZSITHQUOSLWAEEVDROSARLIDEI']
step: 1
max score: 218
['ZSIVYENAROBOENWRUOIMDIAASESI2MLHNOYRDWSHEUL1AIIACLFNEJCREOGRIUVTADAPTEDIETPBINERGTKAGNUFAXEOOLEQTEOT']
step: 2
max score: 222
['ZSIVYEANROBOENWRUOIMDIAASESI2MLHNOYTDWSHEUL1AIIACLFNEJCREOGRIUVTADAPTEDIETPNIBERGTKAGNUFAXEOOLEQTEOR']
step: 3
max score: 232
['YOMBISIOHETIWRG1XJILANUFINAWDVNEORGIAQREEDRAYPPNTIUSAIZSKAET2TOCOEENOALDEEFADMUEURNOOEHITGLVABELTSRC']
step: 4
max score: 232
['YOMBISIOHETIWRG1XJILANUFINAWDVNEORGIAQREEDRAYPPNTIUSAIZSKAET2TOCOEENOALDEEFADMUEURNOOEHITGLVABELTSRC']
step: 5
max score: 251
['ZSIVYENAROBOEQWRUFIMDIAASENI2MLHSOYRDWSHEULAAIIEOGRICVCLONEJURATAD1PTEDIETTBINERGPKAGNUFAXEOOLENTEOT']
step: 6
max score: 251
['ZSIVYENAROBOEQWRUFIMDIAASENI2MLHSOYRDWSHEULAAIIEOGRICVCLONEJURATAD1PTEDIETTBINERGPKAGNUFAXEOOLENTEOT', 'ZSIVYENAROBOEQWRUFIMDIAASENIUMLHSOYRDWSHE2LAAIIEOGRICVCLONEJURATADTP1EDIETTBINERGPKAGNUFAXEOOLENTEOT']


max score: 383
['VIERAPUYTDENIOIGUIOOITENECABZFOKATESICEMGAVOWED2RXUJESSEDNQINORANAPULLREDRRNTEOTGAILAWASHI1EFOHMTYBL']
step: 59
max score: 383
['VIERAPUYTDENIOIGUIOOITENECABZFOKATESICEMGAVOWED2RXUJESSEDNQINORANAPULLREDRRNTEOTGAILAWASHI1EFOHMTYBL']
step: 60
max score: 383
['VIERAPUYTDENIOIGUIOOITENECABZFOKATESICEMGAVOWED2RXUJESSEDNQINORANAPULLREDRRNTEOTGAILAWASHI1EFOHMTYBL']
step: 61
max score: 388
['VIEYOPURTDENIOIGUIOMITENECABZFOKATESICEOGAVOWED2RXUJESSEDNQINARANAPULLREDRRNTEOTGAILAWASHI1EFOHMTYBL']
step: 62
max score: 388
['VIEYOPURTDENIOIGUIOMITENECABZFOKATESICEOGAVOWED2RXUJESSEDNQINARANAPULLREDRRNTEOTGAILAWASHI1EFOHMTYBL']
step: 63
max score: 388
['VIEYOPURTDENIOIGUIOMITENECABZFOKATESICEOGAVOWED2RXUJESSEDNQINARANAPULLREDRRNTEOTGAILAWASHI1EFOHMTYBL']
step: 64
max score: 388
['VIEYOPURTDENIOIGUIOMITENECABZFOKATESICEOGAVOWED2RXUJESSEDNQINARANAPULLREDRRNTEOTGAILAWASHI1EFOHMTYBL']
step: 65
max score: 388
['VIEYOPURTDENIOIGUIOMITENECABZFOKATESICEOGAVOWED2RXUJESSEDNQINARANAPULLREDRRNTEOT

max score: 430
['IVETOPENIAGGUIOMITENECABZFOKATESICETIAVOWED2RXNRTDUJEESEDUQINORANDPULLREYIRNSBOGOILAWASHTR1EFOHMAYAL']
step: 120
max score: 430
['IVETOPENIAGGUIOMITENECABZFOKATESICETIAVOWED2RXNRTDUJEESEDUQINORANDPULLREYIRNSBOGOILAWASHTR1EFOHMAYAL']
step: 121
max score: 430
['IVETOPENIAGGUIOMITENECABZFOKATESICETIAVOWED2RXNRTDUJEESEDUQINORANDPULLREYIRNSBOGOILAWASHTR1EFOHMAYAL']
step: 122
max score: 430
['IVETOPENIAGGUIOMITENECABZFOKATESICETIAVOWED2RXNRTDUJEESEDUQINORANDPULLREYIRNSBOGOILAWASHTR1EFOHMAYAL']
step: 123
max score: 430
['IVETOPENIAGGUIOMITENECABZFOKATESICETIAVOWED2RXNRTDUJEESEDUQINORANDPULLREYIRNSBOGOILAWASHTR1EFOHMAYAL']
step: 124
max score: 430
['IVETOPENIAGGUIOMITENECABZFOKATESICETIAVOWED2RXNRTDUJEESEDUQINORANDPULLREYIRNSBOGOILAWASHTR1EFOHMAYAL']
step: 125
max score: 430
['IVETOPENIAGGUIOMITENECABZFOKATESICETIAVOWED2RXNRTDUJEESEDUQINORANDPULLREYIRNSBOGOILAWASHTR1EFOHMAYAL']
step: 126
max score: 430
['IVETOPENIAGGUIOMITENECABZFOKATESICETIAVOWED2RXNRTDUJEESEDUQINORANDPULLREY

max score: 506
['IVETOPENIAGGEIOMITTIAVOWEDURXURTDNJETRRANDRULLEEYIPNOBFGOILAWASHESUDOSKATESICERNECABZ2QLNO1EFOHMAYAI']
step: 166
max score: 506
['IVETOPENIAGGEIOMITTIAVOWEDURXURTDNJETRRANDRULLEEYIPNOBFGOILAWASHESUDOSKATESICERNECABZ2QLNO1EFOHMAYAI']
step: 167
max score: 506
['IVETOPENIAGGEIOMITTIAVOWEDURXURTDNJETRRANDRULLEEYIPNOBFGOILAWASHESUDOSKATESICERNECABZ2QLNO1EFOHMAYAI']
step: 168
max score: 506
['IVETOPENIAGGEIOMITTIAVOWEDURXURTDNJETRRANDRULLEEYIPNOBFGOILAWASHESUDOSKATESICERNECABZ2QLNO1EFOHMAYAI']
step: 169
max score: 506
['IVETOPENIAGGEIOMITTIAVOWEDURXURTDNJETRRANDRULLEEYIPNOBFGOILAWASHESUDOSKATESICERNECABZ2QLNO1EFOHMAYAI']
step: 170
max score: 506
['IVETOPENIAGGEIOMITTIAVOWEDURXURTDNJETRRANDRULLEEYIPNOBFGOILAWASHESUDOSKATESICERNECABZ2QLNO1EFOHMAYAI']
step: 171
max score: 506
['IVETOPENIAGGEIOMITTIAVOWEDURXURTDNJETRRANDRULLEEYIPNOBFGOILAWASHESUDOSKATESICERNECABZ2QLNO1EFOHMAYAI']
step: 172
max score: 506
['IVETOPENIAGGEIOMITTIAVOWEDURXURTDNJETRRANDRULLEEYIPNOBFGOILAWASHESUDOSKAT

max score: 522
['IVETOPENRAGGEIOMITTIAVOWEDURXURTDNJETGRANDRULLEEYIPNOBFROILAWASHESUDOSKATESICEINECABZ2QLNO1EFOHMAYAI']
step: 230
max score: 522
['IVETOPENRAGGEIOMITTIAVOWEDURXURTDNJETGRANDRULLEEYIPNOBFROILAWASHESUDOSKATESICEINECABZ2QLNO1EFOHMAYAI']
step: 231
max score: 522
['IVETOPENRAGGEIOMITTIAVOWEDURXURTDNJETGRANDRULLEEYIPNOBFROILAWASHESUDOSKATESICEINECABZ2QLNO1EFOHMAYAI']
step: 232
max score: 522
['IVETOPENRAGGEIOMITTIAVOWEDURXURTDNJETGRANDRULLEEYIPNOBFROILAWASHESUDOSKATESICEINECABZ2QLNO1EFOHMAYAI']
step: 233
max score: 522
['IVETOPENRAGGEIOMITTIAVOWEDURXURTDNJETGRANDRULLEEYIPNOBFROILAWASHESUDOSKATESICEINECABZ2QLNO1EFOHMAYAI']
step: 234
max score: 524
['IVETOPENRAGGEIOMITRFAVOWEDURXURTDNJETGRANDRULLEEYIPNOBITOILAWASHESUDOSKATESICEINE1CBZ2QLNOAEFOHMAYAI']
step: 235
max score: 524
['IVETOPENRAGGEIOMITRFAVOWEDURXURTDNJETGRANDRULLEEYIPNOBITOILAWASHESUDOSKATESICEINE1CBZ2QLNOAEFOHMAYAI']
step: 236
max score: 524
['IVETOPENRAGGEIOMITRFAVOWEDURXURTDNJETGRANDRULLEEYIPNOBITOILAWASHESUDOSKAT

max score: 528
['IVETOPENSTDNJETGRANDRUTLEEYFAVOWEDURXURAGGEIOMILRIPNOBITOILAWASHERUDOSKATESICEINE1CBZ2QLNOAEFOHMAYAI', 'IVETOPENSTDNJETGRANDRUTLEEYFAVOWEDURXURAGGEIOMILRIPNOBITOILAWASHERUDOSKATESICEINE1CBZ2QLNOAEFOHMAYAI']
step: 291
max score: 528
['IVETOPENSTDNJETGRANDRUTLEEYFAVOWEDURXURAGGEIOMILRIPNOBITOILAWASHERUDOSKATESICEINE1CBZ2QLNOAEFOHMAYAI', 'IVETOPENSTDNJETGRANDRUTLEEYFAVOWEDURXURAGGEIOMILRIPNOBITOILAWASHERUDOSKATESICEINE1CBZ2QLNOAEFOHMAYAI']
step: 292
max score: 528
['IVETOPENSTDNJETGRANDRUTLEEYFAVOWEDURXURAGGEIOMILRIPNOBITOILAWASHERUDOSKATESICEINE1CBZ2QLNOAEFOHMAYAI', 'IVETOPENSTDNJETGRANDRUTLEEYFAVOWEDURXURAGGEIOMILRIPNOBITOILAWASHERUDOSKATESICEINE1CBZ2QLNOAEFOHMAYAI']
step: 293
max score: 528
['IVETOPENSTDNJETGRANDRUTLEEYFAVOWEDURXURAGGEIOMILRIPNOBITOILAWASHERUDOSKATESICEINE1CBZ2QLNOAEFOHMAYAI', 'IVETOPENSTDNJETGRANDRUTLEEYFAVOWEDURXURAGGEIOMILRIPNOBITOILAWASHERUDOSKATESICEINE1CBZ2QLNOAEFOHMAYAI']
step: 294
max score: 528
['IVETOPENSTDNJETGRANDRUTLEEYFAVOWEDURXURAGGEIOMI

max score: 537
['EVETOPENSTDNJETGRANDRUTFEIYLAVOWEDURXURAGGEIOBILRIPNOBITOILAWASHEREDOSKATESICEINU1CMZ2QLNOAEFOHMAYAI']
step: 323
max score: 537
['EVETOPENSTDNJETGRANDRUTFEIYLAVOWEDURXURAGGEIOBILRIPNOBITOILAWASHEREDOSKATESICEINU1CMZ2QLNOAEFOHMAYAI']
step: 324
max score: 537
['EVETOPENSTDNJETGRANDRUTFEIYLAVOWEDURXURAGGEIOBILRIPNOBITOILAWASHEREDOSKATESICEINU1CMZ2QLNOAEFOHMAYAI']
step: 325
max score: 537
['EVETOPENSTDNJETGRANDRUTFEIYLAVOWEDURXURAGGEIOBILRIPNOBITOILAWASHEREDOSKATESICEINU1CMZ2QLNOAEFOHMAYAI']
step: 326
max score: 540
['EVETOPENSTDRJETGRANCNUTFEIYLAVOWEDURXUNAGGEIOBILRIPROBITOILAWASHEREDOSKATESICEINL1DMZ2QUNOAEFOHMAYAI']
step: 327
max score: 540
['EVETOPENSTDRJETGRANCNUTFEIYLAVOWEDURXUNAGGEIOBILRIPROBITOILAWASHEREDOSKATESICEINL1DMZ2QUNOAEFOHMAYAI']
step: 328
max score: 540
['EVETOPENSTDRJETGRANCNUTFEIYLAVOWEDURXUNAGGEIOBILRIPROBITOILAWASHEREDOSKATESICEINL1DMZ2QUNOAEFOHMAYAI']
step: 329
max score: 540
['EVETOPENSTDRJETGRANCNUTFEIYLAVOWEDURXUNAGGEIOBILRIPROBITOILAWASHEREDOSKAT

In [ ]:
def eval_string_blanks(string,dictionary,maxLen):
    words = set()
    blank1 = set()
    blank2 = set()
    blank12 = set()
    for i in range(len(string)):
        for j in range(min(maxLen,len(string)-i)):
            word = ''.join(string[i:i+j])
            if '1' in word:
                if '2' in word:
                    blank12.add(word)
                else:
                    blank1.add(word)
            elif '2' in word:
                blank2.add(word)
            if word in dictionary:
                words.add(word)
    blank_val = 0
    blank_id = []
    for b1 in tiles.keys():
        blank1_words = set()
        if b1 in ['1','2']:
            continue
            
        for string in blank1:
            word = string.replace('1',b1)
            if (word not in words) & (word in dictionary):
                blank1_words.add(word)
        blank1_val = sum(dictionary[w]-tiles[b1]['points'] for w in blank1_words)
        
        for b2 in tiles.keys():
            blank2_words = set()
            if b2 in ['1','2']:
                continue
                
            for string in blank2:
                word = string.replace('2',b2)
                if (word not in words) & (word not in blank1_words) & (word in dictionary):
                    blank2_words.add(word)
            blank2_val = sum(dictionary[w]-tiles[b2]['points'] for w in blank2_words)
            
            blank12_words = set()
            for string in blank12:
                word = string.replace('1',b1)
                word = word.replace('2',b2)
                if (word in dictionary) & (word not in words) & (word not in blank1_words) & (word not in blank2_words):
                    blank12_words.add(word)
            blank12_val = sum(dictionary[w]-tiles[b1]['points']-tiles[b2]['points'] for w in blank12_words)
            
            current_blank_val = blank1_val + blank2_val + blank12_val
            if current_blank_val > blank_val:
                blank_val = current_blank_val
                blank_id = [b1,b2]
                blank_words = blank1_words | blank2_words | blank12_words
                
            
    val = sum(dictionary[w] for w in words) + blank_val
    return val,words,blank_val,blank_words,blank_id

In [ ]:
def genetic_algorithm_blanks(init_pop,pop_size,n_steps,pnt_mut_rate,swp_mut_rate,keep_best=True):
    
    def reproduce(scores,k):
        scores = np.array(scores,dtype='float')
        scores -= min(scores)
        scores /= sum(scores)
        bins = np.append(0,np.cumsum(scores))
        r = np.random.rand(k)
        idx = np.digitize(r,bins)-1
        return idx
        
    def pnt_mutate(string,pnt_mut_rate):
        n = len(string)
        p = pnt_mut_rate/float(n)
        r = np.random.rand(n)
        for i in range(n):
            if r[i] < p:
                swp_idx = np.random.randint(n)
                swp = string[i]
                string[i] = string[swp_idx]
                string[swp_idx] = swp
                
        return string
    
    def swp_mutate(s,swp_mut_rate):
        if np.random.rand()<swp_mut_rate:
            idx = sorted(np.random.randint(0,len(s),4))
            s = s[0:idx[0]]+s[idx[2]:idx[3]]+s[idx[1]:idx[2]]+s[idx[0]:idx[1]]+s[idx[3]:]
        return s
    
    def score_pop(population):
        score = []
        for indiv in population:
            score.append(eval_string(indiv,dictionary,maxLen)[0])
        return score
    
    population = init_pop
    for step in range(n_steps):
        print 'step: ' + str(step)
        new_pop = []
        score = score_pop(population)
        max_val = max(score)
        max_idx = np.where(np.array(score)==max_val)[0]
        print 'max score: '+ str(max_val)
        best_pop = [population[i] for i in max_idx]
        print [''.join(b) for b in best_pop]
        n_offspring = pop_size
        if keep_best:
            n_offspring -= len(best_pop)
        idx = reproduce(score,n_offspring)
        for i in idx:
            string = population[i][:]
            string = pnt_mutate(string,pnt_mut_rate)
            string = swp_mutate(string,swp_mut_rate)
            new_pop.append(string)
        if keep_best:
            population = best_pop + new_pop
    score = score_pop(pop)
    return new_pop,score
        